Ordering
=====


Warning: This notebook will place live orders
==========================

Use a papertrading account when running this notebook.


In [1]:
%gui qt5

from ib_insync import *
util.useQt()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=13)

In [2]:
contract = Stock('AAPL', 'SMART', 'USD')
ib.qualifyContracts(contract)

order = MarketOrder('BUY', 100)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [3]:
trade = ib.placeOrder(contract, order)

trade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=3, action='BUY', totalQuantity=100), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 28, 864823, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [4]:
trade.log

[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 28, 864823, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 30, 510774, tzinfo=datetime.timezone.utc), status='Submitted', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 30, 686337, tzinfo=datetime.timezone.utc), status='Filled', message='Fill 100.0@148.5')]

``trade`` will also available from ``ib.trades()``,
but not from ``ib.openTrades()`` since it's already filled:

In [5]:
print(trade in ib.trades())
print(trade in ib.openTrades())

True
False


Likewise for ``order``:

In [6]:
print(order in ib.orders())
print(order in ib.openOrders())

True
False


Now let's create a limit order with an unrealistic limit:

In [7]:
limitOrder = LimitOrder('BUY', 100, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=4, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 44, 347445, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

``status`` will change from "PendingSubmit" to "Submitted":

In [8]:
limitTrade.orderStatus.status

'Submitted'

In [9]:
limitTrade in ib.openTrades()

True

Let's modify the limit price and resubmit:

In [10]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=4, action='BUY', totalQuantity=100, lmtPrice=0.1), orderStatus=OrderStatus(status='Submitted', remaining=100.0, permId=1953302638, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 44, 347445, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 44, 508075, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 48, 403895, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify')])

And now cancel it:

In [11]:
ib.cancelOrder(limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=4, action='BUY', totalQuantity=100, lmtPrice=0.1), orderStatus=OrderStatus(status='Submitted', remaining=100.0, permId=1953302638, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 44, 347445, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 44, 508075, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 48, 403895, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify'), TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 50, 910528, tzinfo=datetime.timezone.utc), status='PendingCancel', message='')])

In [12]:
limitTrade.log

[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 44, 347445, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 44, 508075, tzinfo=datetime.timezone.utc), status='Submitted', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 48, 403895, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify'),
 TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 50, 910528, tzinfo=datetime.timezone.utc), status='PendingCancel', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 51, 34940, tzinfo=datetime.timezone.utc), status='Cancelled', message='')]

What's the total of commissions paid today?

In [13]:
sum(fill.commissionReport.commission for fill in ib.fills())

1.0

Some note of warning: reqContractDetails can happily report that a contract is unique,
but with placeOrder() saying that the contract is ambiguous. An example is INTC:

In [13]:
intc = Stock('INTC', 'SMART', 'USD')

print(len(ib.reqContractDetails(intc)))

intcOrder = LimitOrder('BUY', 100, 0.05)
ib.placeOrder(intc, intcOrder)

1


Trade(contract=Stock(symbol='INTC', exchange='SMART', currency='USD'), order=LimitOrder(orderId=6, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 57, 597935, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

ERROR:ib_insync.wrapper:Error 200, reqId 6: The contract description specified for INTC is ambiguous.


In [14]:
ib.trades()[-1].log

[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 57, 597935, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 57, 57, 703069, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 200, reqId 6: The contract description specified for INTC is ambiguous.')]

So it is always a good
idea to a qualify a contract before trading it.

In [15]:
ib.qualifyContracts(intc)

intcOrder = LimitOrder('BUY', 100, 0.05)
ib.placeOrder(intc, intcOrder)

Trade(contract=Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), order=LimitOrder(orderId=8, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 58, 1, 317216, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

In [16]:
ib.cancelOrder(intcOrder)

Trade(contract=Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), order=LimitOrder(orderId=8, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='Submitted', remaining=100.0, permId=1953302639, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 58, 1, 317216, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 58, 1, 444901, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2017, 7, 17, 18, 58, 3, 886497, tzinfo=datetime.timezone.utc), status='PendingCancel', message='')])

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [19]:
%%time
order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while trade.isActive():
    ib.waitOnUpdate()

CPU times: user 9.24 ms, sys: 4.26 ms, total: 13.5 ms
Wall time: 281 ms


What are our positions?

In [20]:
ib.positions()

[Position(account='DU15009', contract=Contract(conId=265598, symbol='AAPL', secType='STK', exchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), position=200.0, avgCost=148.52)]

What's the total of commissions paid today?

In [21]:
sum(fill.commissionReport.commission for fill in ib.fills())

2.0